In [ ]:
# I would like to study the climatology of wind speed and directions
# drawing histograms or hodographs

# Further, this could be linked to convective precipitation

In [2]:
%load_ext autoreload
%autoreload 2

In [ ]:
import numpy as np
import pandas as pd
import glob
import xarray as xr
import matplotlib.pyplot as plt
from scipy import stats

import sys; sys.path.insert(1, '/home/alessandro/scripts')
import os

import geometry as gm
#import conversion as cnv
#from func_statistics import compute_pdf_nbin, slopes_r_p, slope_time, slope_maps
#from coare_functions import *
import meteo
import lcl
import cartopy.crs as ccrs
crs = ccrs.PlateCarree()
#from plotmap import plot_background
from plotdistr import fb_distribution_npoint, perc_distribution, scatterplot_fit, hist2d
import warnings; warnings.filterwarnings("ignore")

In [4]:
# find data
path = '/mercury/Fabien/GS'
wrf_files = path + '/GS700_WRF_3h_surf.nc'
#fname_list = sorted(glob.glob(wrf_files + '*'))

In [ ]:
# open data - HOURLY
ds_tot = xr.open_dataset(wrf_files)    #mfdataset(fname_list, concat_dim='Time', combine='nested')
ds_hourly = ds_tot.sel(time_counter=slice('2005-12-01T00:30:00.000000000', '2006-03-01T00:30:00.000000000'))
del ds_tot

In [5]:
# open data - DAILY
wrf_files = path+'/GS700_WRF_daily_3D.nc'
ds_tot = xr.open_dataset(wrf_files)
ds_daily = ds_tot.sel(time_counter=slice('2005-12-01T00:30:00.000000000', '2006-03-01T00:30:00.000000000'))
del ds_tot

In [6]:
# import mask
if os.path.exists('/home/alessandro/thesis_material/GS_scripts/sea_mask_nobnd.npy'):
    sea_mask_nobnd = np.load('/home/alessandro/thesis_material/GS_scripts/sea_mask_nobnd.npy')

In [7]:
U, V = ds_daily['U'], ds_daily['V']
U_mask, V_mask = (U[:,:,:,0:-1] + U[:,:,:,1:])*0.5, (V[:,:,0:-1,:] + V[:,:,1:,:])*0.5

In [11]:
U_mask, V_mask = U_mask.where(sea_mask_nobnd, drop=False), V_mask.where(sea_mask_nobnd, drop=False)

<xarray.DataArray 'U' (time_counter: 90, lev_M: 49, y_grid_U: 483, x_grid_U: 975)>
array([[[[ 0.29924485,  0.38087714,  0.46772563, ..., -3.5764327 ,
          -3.595641  , -3.6143737 ],
         [ 0.3198558 ,  0.4346887 ,  0.49778956, ..., -3.588078  ,
          -3.5597436 , -3.6077867 ],
         [ 0.37631094,  0.5376844 ,  0.6399512 , ..., -3.6068678 ,
          -3.572431  , -3.6448617 ],
         ...,
         [ 0.7602253 ,  0.61476904,  0.47694314, ..., -7.0305896 ,
          -7.078514  , -7.163665  ],
         [ 0.7986052 ,  0.6691514 ,  0.5535791 , ..., -7.109935  ,
          -7.107235  , -7.147789  ],
         [ 0.8369251 ,  0.74562186,  0.6520908 , ..., -7.1287427 ,
          -7.136243  , -7.1427383 ]],

        [[ 0.34297538,  0.41920856,  0.49942166, ..., -3.929379  ,
          -3.948176  , -3.966906  ],
         [ 0.36670113,  0.48998713,  0.56729394, ..., -3.9313273 ,
          -3.9189148 , -3.9739718 ],
         [ 0.4179178 ,  0.6036333 ,  0.74101377, ..., -3.9464293 ,
          -3.9232495 , -4.008015  ],
...
         [16.390335  , 16.335907  , 16.33513   , ..., 16.484352  ,
          16.482851  , 16.557056  ],
         [16.389416  , 16.357178  , 16.361622  , ..., 16.534021  ,
          16.530025  , 16.57003   ],
         [16.402073  , 16.396835  , 16.39409   , ..., 16.574799  ,
          16.58104   , 16.586502  ]],

        [[32.86326   , 32.906303  , 32.950813  , ..., 34.0573    ,
          34.04169   , 34.025486  ],
         [32.85266   , 32.889923  , 32.9431    , ..., 34.005745  ,
          33.982876  , 33.970406  ],
         [32.85121   , 32.8914    , 32.948444  , ..., 33.927162  ,
          33.90735   , 33.910587  ],
         ...,
         [20.532068  , 20.460096  , 20.444477  , ..., 19.856693  ,
          19.845417  , 19.895676  ],
         [20.5473    , 20.502491  , 20.492691  , ..., 19.879368  ,
          19.86739   , 19.889473  ],
         [20.570765  , 20.555143  , 20.538925  , ..., 19.893755  ,
          19.890415  , 19.887417  ]]]], dtype=float32)
Coordinates:
    nav_lat_grid_U  (y_grid_U, x_grid_U) float32 33.87 33.87 ... 43.01 43.01
  * lev_M           (lev_M) float32 1.0 2.0 3.0 4.0 5.0 ... 46.0 47.0 48.0 49.0
    time_centered   (time_counter) datetime64[ns] 2005-12-01T12:00:00 ... 200...
  * time_counter    (time_counter) datetime64[ns] 2005-12-01T12:00:00 ... 200...
Dimensions without coordinates: y_grid_U, x_grid_U